# Convolutional Neural Networks

## Project: Write an Algorithm for Landmark Classification

### A simple app

In this notebook we build a very simple app that uses our exported model.

> <img src="static_images/icons/noun-info-2558213.png" alt="?" style="width:25px"/> Note how we are not importing anything from our source code (we do not use any module from the ``src`` directory). This is because the exported model, differently from the model weights, is a standalone serialization of our model and therefore it does not need anything else. You can ship that file to anybody, and as long as they can import ``torch``, they will be able to use your model. This is very important for releasing pytorch models to production.

### Test your app
Go to a search engine for images (like Google Images) and search for images of some of the landmarks, like the Eiffel Tower, the Golden Gate Bridge, Machu Picchu and so on. Save a few examples locally, then upload them to your app to see how your model behaves!

The app will show the top 5 classes that the model think are most relevant for the picture you have uploaded

In [2]:
# Install requirements
!pip install -r requirements.txt | grep -v "already satisfied"

  Using cached ipywidgets-8.1.1-py3-none-any.whl (139 kB)
  Using cached livelossplot-0.5.5-py3-none-any.whl (22 kB)
  Using cached pytest-7.4.2-py3-none-any.whl (324 kB)
  Using cached nbconvert-7.9.2-py3-none-any.whl (256 kB)
  Using cached widgetsnbextension-4.0.9-py3-none-any.whl (2.3 MB)
  Using cached jupyterlab_widgets-3.0.9-py3-none-any.whl (214 kB)
  Using cached iniconfig-2.0.0-py3-none-any.whl (5.9 kB)
  Using cached exceptiongroup-1.1.3-py3-none-any.whl (14 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached bleach-6.1.0-py3-none-any.whl (162 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
  Using cached jupyterlab_pygments-0.2.2-py2.py3-none-any.whl (21 kB)
  Using cached mistune-3.0.2-py3-none-any.whl (47 kB)
  Using cached nbclient-0.8.0-py3-none-any.whl (73 kB)
  Using cached nbformat-5.9.2-py3-none-any.whl (77 kB)
  Using cached pandocfilters-1.5.0-py2.py3-none-any.whl (8.7 kB)
  Using cached tinycss2-1.2.1-py3-none-any.wh

In [3]:
!pip install --upgrade jupyter

  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
  Using cached notebook-7.0.6-py3-none-any.whl (4.0 MB)
  Using cached qtconsole-5.4.4-py3-none-any.whl (121 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl (24 kB)
  Using cached jupyter_server-2.8.0-py3-none-any.whl (377 kB)
  Using cached jupyterlab_server-2.25.0-py3-none-any.whl (57 kB)
  Using cached jupyterlab-4.0.7-py3-none-any.whl (9.2 MB)
  Using cached notebook_shim-0.2.3-py3-none-any.whl (13 kB)
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
  Using cached QtPy-2.4.0-py3-none-any.whl (93 kB)
  Using cached anyio-4.0.0-py3-none-any.whl (83 kB)
  Using cached argon2_cffi-23.1.0-py3-none-any.whl (15 kB)
  Using cached jupyter_events-0.8.0-py3-none-any.whl (18 kB)
  Using cached jupyter_server_terminals-0.4.4-py3-none-any.whl (13 kB)
  Using cached overrides-7.4.0-py3-none-any.whl (17 kB)
  Using cached prometheus_client-0.17.1-py3-none-any.whl (60 kB)
  Using cached Send2Trash-1.8.2-py3-non

In [4]:
# Enable widgets
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


Now restart your kernel and continue

In [4]:
from ipywidgets import VBox, Button, FileUpload, Output, Label
from PIL import Image
from IPython.display import display
import io
import numpy as np
import torchvision
import torchvision.transforms as T
import torch

# Decide which model you want to use among the ones exported
with open("checkpoints/transfer_exported.pt", "rb") as f:
    
learn_inf = torch.jit.load("checkpoints/transfer_exported.pt", map_location=torch.device('cpu'))# YOUR CODE HERE

def on_click_classify(change):

    # Load image that has been uploaded
    fn = io.BytesIO(btn_upload.value[0]['content'])

    img = Image.open(fn)
    img.load()

    # Let's clear the previous output (if any)
    out_pl.clear_output()

    # Display the image
    with out_pl:

        ratio = img.size[0] / img.size[1]
        c = img.copy()
        c.thumbnail([ratio * 200, 200])
        display(c)

    # Transform to tensor
    timg = T.ToTensor()(img).unsqueeze_(0)

    # Calling the model
    softmax = learn_inf(timg).data.cpu().numpy().squeeze()
    
    # Get the indexes of the classes ordered by softmax
    # (larger first)
    idxs = np.argsort(softmax)[::-1]
    
    # Loop over the classes with the largest softmax
    for i in range(5):
        # Get softmax value
        p = softmax[idxs[i]]
    
        # Get class name
        landmark_name = learn_inf.class_names[idxs[i]]
        
        labels[i].value = f"{landmark_name} (prob: {p:.2f})"


# Putting back btn_upload to a widget for next cell
btn_upload = FileUpload()

btn_run = Button(description="Classify")
btn_run.on_click(on_click_classify)

labels = []
for _ in range(5):
    labels.append(Label())

out_pl = Output()
out_pl.clear_output()

wgs = [Label("Please upload a picture of a landmark"), btn_upload, btn_run, out_pl]
wgs.extend(labels)

VBox(wgs)

## (optional) Standalone app or web app

You can run this notebook as a standalone app on your computer by following these steps:

1. Download this notebook in a directory on your machine
2. Download the model export (for example, ``checkpoints/transfer_exported.pt``) in a subdirectory called ``checkpoints`` within the directory where you save the app.ipynb notebook
3. Install voila if you don't have it already (``pip install voila``)
4. Run your app: ``voila app.ipynb --show_tracebacks=True``
5. Customize your notebook to make your app prettier and rerun voila

You can also deploy this app as a website using Binder: https://voila.readthedocs.io/en/stable/deploy.html#deployment-on-binder

# Create your submission archive

Now that you are done with your project, please run the following cell. It will generate a file containing all the code you have written, as well as the notebooks. Please submit that file to complete your project

In [2]:
!python src/create_submit_pkg.py

executing: jupyter nbconvert --to html transfer_learning.ipynb
[NbConvertApp] Converting notebook transfer_learning.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 2 image(s).
[NbConvertApp] Writing 482857 bytes to transfer_learning.html
executing: jupyter nbconvert --to html app.ipynb
[NbConvertApp] Converting notebook app.ipynb to html
[NbConvertApp] Writing 315085 bytes to app.html
executing: jupyter nbconvert --to html cnn_from_scratch.ipynb
[NbConvertApp] Converting notebook cnn_from_scratch.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 3 image(s).
[NbConvertApp] Writing 1457148 bytes to cnn_from_scratch.html
Adding files to submission_2023-10-20T11h02m.zip
src/model.py
src/transfer.py
src/helpers.py
src/model.prv.py
src/optimization.py
src/data.py
src/train.py
src/predictor.py
src/__init__.py
src/create_submit_pkg.py
transfer_learning.ipynb
app.ipynb
cnn_from_scratch.ipynb
app.html
cnn_from_scratch.html
transfer_learning.html

-------